# Stock Comparison Tool

Trying to implement something similar to https://www.barchart.com/myBarchart/quotes/SPY/interactive-chart but in Gradio.

The code in the first code block of this notebook is executed when the `app.py` file is run. This notebook is used for the simplicity of interactive development and testing.

In [1]:
from datetime import timedelta

import gradio as gr
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import yfinance as yf
from dash import Dash, Input, Output, State, dcc, html

# retrieve historical stock prices
tickers = ["AAPL", "MSFT", "GOOGL"]
df = (
    yf.download(
        tickers,
        interval="1d",
        period="max",
        progress=False,
    )
    .Close.bfill()
    .ffill()
)
df

Ticker,AAPL,GOOGL,MSFT
Date,,,
1980-12-12,0.098834,2.501941,0.059947
1980-12-15,0.093678,2.501941,0.059947
1980-12-16,0.086802,2.501941,0.059947
1980-12-17,0.088951,2.501941,0.059947
1980-12-18,0.091530,2.501941,0.059947
...,...,...,...
2024-12-27,255.589996,192.759995,430.529999
2024-12-30,252.199997,191.240005,424.829987
2024-12-31,250.419998,189.300003,421.500000


### Gradio app -- this works but is slow to update

In [2]:
def plot_asset_prices(period, shift):
    end_date = df.index[-1] - timedelta(days=shift)
    start_date = end_date - timedelta(days=period)
    df_normalized = df[(df.index >= start_date) & (df.index <= end_date)]
    df_normalized = df_normalized / df_normalized.iloc[0] - 1
    df_normalized.reset_index(names="Date", inplace=True)
    return gr.LinePlot(
        value=pd.melt(df_normalized, id_vars=["Date"], var_name="Asset", value_name="Price"),
        x="Date",
        y="Price",
        color="Asset",
        title="Normalized Asset Prices",
        y_title="Relative Change",
        x_label_angle=45,
        # height=600,
    )


with gr.Blocks() as demo:
    plot = plot_asset_prices(365, 0)
    with gr.Row():
        period = gr.Radio(
            choices=[
                ("5y", 5 * 365),
                ("3y", 2 * 365),
                ("2y", 2 * 365),
                ("1y", 365),
                ("6mo", 182),
                ("1mo", 30),
                ("1w", 7),
            ],
            value=365,
            label="Period",
        )
        shift = gr.Slider(minimum=0, maximum=365, value=0, label="End Date")
    period.change(plot_asset_prices, inputs=[period, shift], outputs=plot)
    shift.change(plot_asset_prices, inputs=[period, shift], outputs=plot)

demo.launch()

* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


### Plotly app -- a bit better

In [3]:
class NormalizedAssetPricesApp:

    def __init__(self, prices):
        self.prices = prices / prices.loc[prices.index[0]]
        self.timestamps = self.prices.index
        self.setup_app()

    def setup_app(self):
        self.app = Dash(__name__)
        self.app.layout = html.Div(
            [dcc.Graph(id="plotly-normalized-asset-prices", figure=self.plot_prices())]
        )

        @self.app.callback(
            Output("plotly-normalized-asset-prices", "figure"),
            Input("plotly-normalized-asset-prices", "relayoutData"),
            State("plotly-normalized-asset-prices", "figure"),
        )
        def update_plot(relayout_data, current_figure):
            """Preserve whichever x-range the user has set by zoom/pan or slider."""
            t_range = self.get_x_range(relayout_data, current_figure)
            return self.plot_prices(t_range)

    def run(self, **kwargs):
        self.app.run_server(**kwargs)

    # TODO: only update if the range changed
    def plot_prices(self, t_range=[None, None]):
        # idx = self.timestamps.get_indexer([t_range[0]], method="nearest")[0] if all(t_range) else 0
        # price = self.prices.loc[self.timestamps[idx]]
        # prices_normalized = self.prices / price - 1
        # fig = go.Figure()

        # normalize prices in t_range
        i, j = self.timestamps.get_indexer(t_range, method="nearest") if all(t_range) else [0, -1]
        t0, t1 = self.timestamps[i], self.timestamps[j]
        price = self.prices.loc[t0]
        prices_normalized = np.nan * self.prices
        prices_normalized.loc[t0:t1] = self.prices[t0:t1] / price - 1
        fig = go.Figure()

        # rangeslider plot
        for asset in self.prices.columns:
            fig.add_trace(
                go.Scatter(
                    x=self.timestamps,
                    y=self.prices[asset],
                    xaxis="x1",
                    yaxis="y1",
                    showlegend=False,
                )
            )

        # main plot
        for asset in prices_normalized.columns:
            fig.add_trace(
                go.Scatter(
                    x=self.timestamps,
                    y=prices_normalized[asset],
                    name=asset,
                    xaxis="x2",
                    yaxis="y2",
                )
            )

        # configure axes
        xaxis1_dict = dict(rangeslider=dict(visible=True))
        xaxis2_dict = dict(matches="x1")
        if all(t_range):
            xaxis1_dict["range"] = t_range
            xaxis2_dict["range"] = t_range
        yaxis1_dict = dict(type="log", showticklabels=False)
        yaxis2_dict = dict(title="Relative Change")

        fig.update_layout(
            xaxis1=xaxis1_dict,
            yaxis1=yaxis1_dict,
            xaxis2=xaxis2_dict,
            yaxis2=yaxis2_dict,
            uirevision="constant-key",  # prevent resets from the xrange compression
        )

        return fig

    def get_x_range(self, relayout_data, current_figure, x_range=[None, None]):
        if relayout_data:
            # Check the main axis first (common when user zooms the top plot)
            if "xaxis2.range[0]" in relayout_data:
                x_range = [relayout_data["xaxis2.range[0]"], relayout_data["xaxis2.range[1]"]]

            # If not found, check for "xaxis1.range[0]" (if Plotly is labeling the slider that way)
            elif "xaxis1.range[0]" in relayout_data:
                x_range = [relayout_data["xaxis1.range[0]"], relayout_data["xaxis1.range[1]"]]

            # If still not found, it might be "xaxis.range[0]" for the slider
            elif "xaxis.range[0]" in relayout_data:
                x_range = [relayout_data["xaxis.range[0]"], relayout_data["xaxis.range[1]"]]

            # If no recognized keys are present, we try to read from the figure layout
            else:
                layout = current_figure["layout"]
                if layout.get("xaxis2", {}).get("range"):
                    x_range = layout["xaxis2"]["range"]
                elif layout.get("xaxis1", {}).get("range"):
                    x_range = layout["xaxis1"]["range"]

        # No relayout_data at all, read from the figure layout
        else:
            layout = current_figure["layout"]
            if layout.get("xaxis2", {}).get("range"):
                x_range = layout["xaxis2"]["range"]
            elif layout.get("xaxis1", {}).get("range"):
                x_range = layout["xaxis1"]["range"]

        return x_range

In [4]:
app = NormalizedAssetPricesApp(df)
app.run(debug=True)